# 05_noise_robustness.ipynb — sweep shots & noise for QSVM + VQC

# Cell 0 — perf env (keep structure; nicer output)

In [ ]:
# Configure deterministic thread counts and lightweight timing utilities for the sweep
import os
os.environ.setdefault("OMP_NUM_THREADS", "8")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "8")
os.environ.setdefault("MKL_NUM_THREADS", "8")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "8")
print("BLAS threads:",
      os.environ.get("OMP_NUM_THREADS"),
      os.environ.get("OPENBLAS_NUM_THREADS"),
      os.environ.get("MKL_NUM_THREADS"),
      os.environ.get("NUMEXPR_NUM_THREADS"))

import time, json
from contextlib import contextmanager
from collections import defaultdict

class PhaseTimer:
    def __init__(self): self.t = defaultdict(float)
    @contextmanager
    def timed(self, key):
        t0 = time.perf_counter()
        yield
        self.t[key] += time.perf_counter() - t0
    def add(self, key, seconds): self.t[key] += seconds
    def to_dict(self): return dict(self.t)

def pretty_seconds(sec):
    return f"{sec/60:.1f} min" if sec >= 60 else f"{sec:.1f} s"

# Cell 1 — imports, dirs, RunJournal (step logs saved to Markdown + JSON)

In [ ]:
# Core imports and directory setup; include robust template import for older/newer PennyLane versions
from pathlib import Path
import itertools, warnings, numpy as np, pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix,
    balanced_accuracy_score, matthews_corrcoef, classification_report, average_precision_score
)

warnings.filterwarnings("ignore")

# PennyLane template import (version-robust)
try:
    BasicEntanglerLayers = qml.BasicEntanglerLayers
except AttributeError:
    from pennylane.templates.layers import BasicEntanglerLayers

ROOT = Path(".")
PROCESSED = ROOT/"data/processed"
RESULTS = ROOT/"results"
(RESULTS / "metrics").mkdir(parents=True, exist_ok=True)
(RESULTS / "kernels").mkdir(parents=True, exist_ok=True)
(RESULTS / "cache").mkdir(parents=True, exist_ok=True)
(RESULTS / "plots").mkdir(parents=True, exist_ok=True)
(RESULTS / "logs").mkdir(parents=True, exist_ok=True)

np.random.seed(17); pnp.random.seed(17)

# --- Run Journal: print + save Markdown/JSON for documentation ---
class RunJournal:
    def __init__(self): self.events = []
    def log(self, step, status, message, **extras):
        row = {"ts": time.strftime("%Y-%m-%d %H:%M:%S"), "step": step, "status": status, "message": message}
        row.update(extras)
        self.events.append(row)
        sym = "✅" if status == "ok" else ("⚠️" if status == "warn" else "❌")
        print(f"{sym} [{step}] {message}")
    def df(self): return pd.DataFrame(self.events)
    def save(self, basepath: Path):
        df = self.df()
        md = ["| ts | step | status | message |", "|---|---|---|---|"]
        for _, r in df.iterrows():
            md.append(f"| {r.ts} | {r.step} | {r.status} | {r.message} |")
        (basepath.with_suffix(".md")).write_text("\n".join(md), encoding="utf-8")
        (basepath.with_suffix(".json")).write_text(df.to_json(orient="records", indent=2), encoding="utf-8")
        print(f"📝 Saved journal:\n  - {basepath.with_suffix('.md')}\n  - {basepath.with_suffix('.json')}")

J = RunJournal()
J.log("init", "ok", "Noise robustness sweep notebook initialized; directories ready.")

# Cell 2 — knobs (speed vs accuracy; clearly printed)

In [ ]:
# Define experiment size/quality presets controlling data subsampling, anchor count, shots, and noise grids
PRESET = "turbo"  # Options: "turbo" (fastest), "fast", "full"

if PRESET == "turbo":
    D = 6; MAX_TR = 160; M_ANCHORS = 96
    shots_grid  = [256]
    pflip_grid  = [0.0, 0.01]
    pdepol_grid = [0.0]
elif PRESET == "fast":
    D = 6; MAX_TR = 220; M_ANCHORS = 128
    shots_grid  = [256, 512]
    pflip_grid  = [0.0, 0.01]
    pdepol_grid = [0.0, 0.01]
else:  # full
    D = 6; MAX_TR = 300; M_ANCHORS = 192
    shots_grid  = [512, 2000]
    pflip_grid  = [0.0, 0.01]
    pdepol_grid = [0.0, 0.01]

SWEEP = list(itertools.product(shots_grid, pflip_grid, pdepol_grid))
print(f"Preset={PRESET} | Sweep size: {len(SWEEP)} configs")
J.log("config", "ok", f"Preset={PRESET}, D={D}, MAX_TR={MAX_TR}, M_ANCHORS={M_ANCHORS}, sweep={len(SWEEP)}")

# Cell 3 — load & PCA (multi-dataset aware; logs outcomes & problems)

In [ ]:
# Load feature encodings; apply PCA + scaling; optionally truncate training set for speed
enc_path = PROCESSED/"encodings_all.npz" if (PROCESSED/"encodings_all.npz").exists() else PROCESSED/"encodings.npz"
spl_path = PROCESSED/"splits_pooled.json" if (PROCESSED/"splits_pooled.json").exists() else PROCESSED/"splits.json"

data = np.load(enc_path, allow_pickle=True)
import json
with open(spl_path) as f: SPL = json.load(f)
J.log("load", "ok", f"Loaded encodings from {enc_path.name} and splits from {spl_path.name}")

y = data["y"].astype(int)
X_kmer = data["kmer"].astype(np.float32)
tr_idx = np.array(SPL["train"]); va_idx = np.array(SPL["val"]); te_idx = np.array(SPL["test"])

pca = PCA(n_components=D, random_state=17)
scaler = StandardScaler(with_mean=True, with_std=True)

X_tr_pca = pca.fit_transform(X_kmer[tr_idx])
X_va_pca = pca.transform(X_kmer[va_idx])
X_te_pca = pca.transform(X_kmer[te_idx])

Xtr = scaler.fit_transform(X_tr_pca).astype(np.float32)
Xva = scaler.transform(X_va_pca).astype(np.float32)
Xte = scaler.transform(X_te_pca).astype(np.float32)

ytr, yva, yte = y[tr_idx], y[va_idx], y[te_idx]
sel = np.arange(min(MAX_TR, len(Xtr)))
Xtr_s, ytr_s = Xtr[sel], ytr[sel]

print("Shapes:", Xtr.shape, Xva.shape, Xte.shape, "| train subset:", Xtr_s.shape)
J.log("pca_scale", "ok", f"PCA D={D} (explained_var={pca.explained_variance_ratio_.sum():.3f}); subset_train={Xtr_s.shape}")
if len(np.unique(ytr_s)) < 2:
    J.log("limitation", "warn", "Train subset became single-class; some metrics (AUC) may be undefined.")

# Cell 4 — VQC helpers (statevector + MC noisy variant)

In [ ]:
# Statevector VQC (clean + Monte Carlo noisy variants) for probability estimation

def _make_sv_device():
    try:
        dev = qml.device("lightning.qubit", wires=D, shots=None)
        J.log("device", "ok", f"Using lightning.qubit for VQC (D={D})")
        return dev
    except Exception as e:
        J.log("device", "warn", f"lightning.qubit unavailable ({e}); falling back to default.qubit")
        return qml.device("default.qubit",  wires=D, shots=None)

def _vqc_layer_clean(x, w):
    qml.AngleEmbedding(x, wires=range(D), rotation="Y")
    BasicEntanglerLayers(w[None, :], wires=range(D))

_dev_vqc = _make_sv_device()

@qml.qnode(_dev_vqc, interface=None)
def _vqc_clean(x, weights):
    for l in range(weights.shape[0]):
        _vqc_layer_clean(x, weights[l])
    return qml.expval(qml.PauliZ(0))

# Monte Carlo noise mask sampling (Pauli + bit flips)
def _sample_noise_masks(rng, pflip, pdepol):
    flips = (rng.random(D) < pflip).astype(np.int8)
    pa = np.zeros(D, dtype=np.int8)
    mask = rng.random(D) < pdepol
    if mask.any():
        pa[mask] = rng.integers(1, 4, size=int(mask.sum()))  # 1:X,2:Y,3:Z
    return flips, pa

def _apply_noise(pa, flips):
    for i in range(D):
        lab = int(pa[i])
        if lab == 1: qml.PauliX(i)
        elif lab == 2: qml.PauliY(i)
        elif lab == 3: qml.PauliZ(i)
    if flips.any():
        for i in range(D):
            if flips[i]: qml.PauliX(i)

@qml.qnode(_dev_vqc, interface=None)
def _vqc_noisy_once(x, weights, flips, pa):
    for l in range(weights.shape[0]):
        qml.AngleEmbedding(x, wires=range(D), rotation="Y")
        _apply_noise(pa, flips)  # Insert noise between data re-upload and entangling layer
        BasicEntanglerLayers(weights[l][None, :], wires=range(D))
    return qml.expval(qml.PauliZ(0))

# Cell 5 — Load trained VQC weights (from notebook 04; log if missing)

In [ ]:
# Load pretrained VQC weights produced in notebook 04 (required for reuse in robustness study)
W_PATH = RESULTS / "vqc_weights.npy"
if not W_PATH.exists():
    J.log("weights", "fail", "Missing vqc_weights.npy (run 04_quantum_vqc.ipynb first).")
    raise FileNotFoundError("Run 04_quantum_vqc.ipynb first to save weights.")
weights = pnp.array(np.load(W_PATH), requires_grad=False)
J.log("weights", "ok", f"Loaded VQC weights with shape {weights.shape}")
weights.shape  # (L, D)

# Cell 6 — VQC predict & full metrics (incl. PR-AUC, Spec, MCC)

In [ ]:
# Generate VQC probabilities under optional Pauli + bit-flip noise via light MC sampling; derive split metrics
def vqc_predict_proba(X, weights, pflip=0.0, pdepol=0.0, shots=0):
    w = np.asarray(weights, dtype=float)
    rng = np.random.default_rng(123)
    out = []

    if pflip == 0.0 and pdepol == 0.0:  # Pure path
        for xi in X:
            m = float(_vqc_clean(xi, w))
            out.append((1.0 + m)/2.0)
    else:  # Monte Carlo noisy approximation
        S = max(16, shots // 32) if shots else 32
        for xi in X:
            acc = 0.0
            for _ in range(S):
                flips, pa = _sample_noise_masks(rng, pflip, pdepol)
                m = float(_vqc_noisy_once(xi, w, flips, pa))
                acc += (1.0 + m)/2.0
            out.append(acc / S)

    return np.clip(np.array(out, dtype=float), 1e-6, 1.0-1e-6)

def extended_metrics_from_probs(p, y, split, thr=0.5):
    """Extended metrics + confusion matrix, with guards for undefined AUC."""
    yhat = (p >= thr).astype(int)
    acc = accuracy_score(y, yhat)
    prec, rec, f1, _ = precision_recall_fscore_support(y, yhat, average="binary", zero_division=0)
    try:
        auc = roc_auc_score(y, p)
    except ValueError:
        auc = float("nan")
    try:
        ap = average_precision_score(y, p)
    except Exception:
        ap = float("nan")
    cm = confusion_matrix(y, yhat, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    tnr = tn / (tn + fp) if (tn + fp) else float("nan")            # Specificity
    bal = balanced_accuracy_score(y, yhat)
    mcc = matthews_corrcoef(y, yhat) if len(np.unique(y)) == 2 else float("nan")
    rep = classification_report(y, yhat, output_dict=True, zero_division=0)
    return (
        dict(split=split, acc=acc, prec=prec, rec=rec, f1=f1, auc=auc,
             pr_auc=ap, balanced_acc=bal, specificity=tnr, mcc=mcc, thr=thr),
        cm,
        rep
    )

def save_cm_csv(cm, out_csv, normalized=False):
    arr = cm.astype(np.float64)
    if normalized:
        rs = arr.sum(axis=1, keepdims=True)
        arr = np.divide(arr, np.where(rs == 0, 1, rs))
    df = pd.DataFrame(arr, index=["true_0","true_1"], columns=["pred_0","pred_1"])
    df.to_csv(out_csv, index=True)

# Cell 7 — Kernel helpers (pure + MC adjoint) & Nyström utilities (readable)

In [ ]:
# Quantum kernel helper functions: pure (state overlap) and noisy (MC state sampling) + Nyström utilities

def _entangle_ring(ws):
    N = len(ws)
    for i in range(N):
        qml.CZ(wires=[ws[i], ws[(i+1)%N]])

def save_npz(path, **arrays): np.savez_compressed(path, **arrays)
def load_npz(path): return dict(np.load(path, allow_pickle=True)) if Path(path).exists() else None

def normalize_block(K, da, db):
    da = np.where(da <= 1e-12, 1e-12, da)
    db = np.where(db <= 1e-12, 1e-12, db)
    return np.clip(K, 0.0, 1.0) / (np.sqrt(np.outer(da, db)) + 1e-12)

# Pure (noiseless) kernel via explicit state construction + amplitude inner products
def make_pure_state_getter():
    try:
        dev = qml.device("lightning.qubit", wires=D, shots=None)
        J.log("device", "ok", f"Using lightning.qubit for pure-kernel states (D={D})")
    except Exception as e:
        J.log("device", "warn", f"lightning.qubit unavailable ({e}); fallback to default.qubit")
        dev = qml.device("default.qubit", wires=D, shots=None)

    @qml.qnode(dev)
    def phi(x):
        qml.AngleEmbedding(x, wires=range(D), rotation="Y")
        _entangle_ring(list(range(D)))
        return qml.state()
    return phi

def states_batch(X, get_state, dtype=np.complex64):
    return np.stack([get_state(x) for x in X]).astype(dtype, copy=False)

def kernel_from_states(SA, SB):
    M = SA @ SB.conj().T
    return np.abs(M)**2

# Noisy kernel estimation using pairwise independent noise masks (MC averaged)
def _noise_masks(rng, pflip, pdepol):
    flips = (rng.random(D) < pflip).astype(np.int8)
    pa = np.zeros(D, dtype=np.int8)
    mask = rng.random(D) < pdepol
    if mask.any():
        pa[mask] = rng.integers(1, 4, size=int(mask.sum()))
    return flips, pa

def _build_noisy_state_qnode():
    try:
        dev = qml.device("lightning.qubit", wires=D, shots=None)
    except Exception:
        dev = qml.device("default.qubit", wires=D, shots=None)

    def _apply(pa, flips):
        for i in range(D):
            lab = int(pa[i])
            if lab == 1: qml.PauliX(i)
            elif lab == 2: qml.PauliY(i)
            elif lab == 3: qml.PauliZ(i)
        if flips.any():
            for i in range(D):
                if flips[i]: qml.PauliX(i)

    @qml.qnode(dev)
    def psi(x, flips, pa):
        qml.AngleEmbedding(x, wires=range(D), rotation="Y")
        _apply(pa, flips)  # pre-entangle
        _entangle_ring(list(range(D)))
        _apply(pa, flips)  # post-entangle
        return qml.state()
    return psi

def _states_noise_batch(X, flips, pa, psi):
    return np.stack([psi(x, flips, pa) for x in X]).astype(np.complex64, copy=False)

def mc_blocks_via_states(Xtr_s, Xva, Xte, A, pflip, pdepol, S=24, seed=123):
    """Monte Carlo average of kernel blocks using independent left/right noise masks."""
    psi = _build_noisy_state_qnode()
    rng = np.random.default_rng(seed)
    M = len(A)

    K_MM = np.zeros((M, M), dtype=float)
    K_trM = np.zeros((len(Xtr_s), M), dtype=float)
    K_vaM = np.zeros((len(Xva),   M), dtype=float)
    K_teM = np.zeros((len(Xte),   M), dtype=float)
    d_M  = np.zeros(M, dtype=float)
    d_tr = np.zeros(len(Xtr_s), dtype=float)
    d_va = np.zeros(len(Xva),   dtype=float)
    d_te = np.zeros(len(Xte),   dtype=float)

    for _ in range(S):
        flipsL, paL = _noise_masks(rng, pflip, pdepol)
        flipsR, paR = _noise_masks(rng, pflip, pdepol)

        A_L   = _states_noise_batch(A,     flipsL, paL, psi)
        A_R   = _states_noise_batch(A,     flipsR, paR, psi)
        tr_L  = _states_noise_batch(Xtr_s, flipsL, paL, psi)
        tr_R  = _states_noise_batch(Xtr_s, flipsR, paR, psi)
        va_L  = _states_noise_batch(Xva,   flipsL, paL, psi)
        va_R  = _states_noise_batch(Xva,   flipsR, paR, psi)
        te_L  = _states_noise_batch(Xte,   flipsL, paL, psi)
        te_R  = _states_noise_batch(Xte,   flipsR, paR, psi)

        K_MM  += np.abs(A_L  @ A_R.conj().T)**2
        K_trM += np.abs(tr_L @ A_R.conj().T)**2
        K_vaM += np.abs(va_L @ A_R.conj().T)**2
        K_teM += np.abs(te_L @ A_R.conj().T)**2

        d_M  += np.abs(np.sum(A_L  * A_R.conj(), axis=1))**2
        d_tr += np.abs(np.sum(tr_L * tr_R.conj(), axis=1))**2
        d_va += np.abs(np.sum(va_L * va_R.conj(), axis=1))**2
        d_te += np.abs(np.sum(te_L * te_R.conj(), axis=1))**2

    invS = 1.0/float(S)
    return (K_MM*invS, K_trM*invS, K_vaM*invS, K_teM*invS,
            d_M*invS,   d_tr*invS,  d_va*invS,  d_te*invS)

# Cell 8 — VQC sweep (timed; val-optimal threshold; full metrics; CSV)

In [ ]:
# Sweep VQC under varying noise + shot configurations; collect split metrics with timing
def plot_roc(y_true, y_prob, title, out_png):
    from sklearn.metrics import roc_curve, auc
    try:
        fpr, tpr, _ = roc_curve(y_true, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
        plt.plot([0,1],[0,1], linestyle="--")
        plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(title); plt.legend(loc="lower right"); plt.tight_layout()
        plt.savefig(out_png, dpi=150); plt.close()
    except Exception as e:
        J.log("plot", "warn", f"VQC ROC plot skipped: {e}")

def plot_pr(y_true, y_prob, title, out_png):
    from sklearn.metrics import precision_recall_curve, average_precision_score
    try:
        prec, rec, _ = precision_recall_curve(y_true, y_prob)
        ap = average_precision_score(y_true, y_prob)
        plt.figure()
        plt.plot(rec, prec, label=f"AP={ap:.3f}")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(title); plt.legend(loc="lower left"); plt.tight_layout()
        plt.savefig(out_png, dpi=150); plt.close()
    except Exception as e:
        J.log("plot", "warn", f"VQC PR plot skipped: {e}")

timer_v = PhaseTimer()
rows = []

with timer_v.timed(f"VQC_sweep_total_{len(SWEEP)}"):
    for shots, pflip, pdepol in SWEEP:
        cfg = f"shots={shots}, pflip={pflip}, pdepol={pdepol}"
        with timer_v.timed(f"VQC_forward_{shots}_{pflip}_{pdepol}"):
            p_tr = vqc_predict_proba(Xtr, weights, pflip=pflip, pdepol=pdepol, shots=shots)
            p_va = vqc_predict_proba(Xva, weights, pflip=pflip, pdepol=pdepol, shots=shots)
            p_te = vqc_predict_proba(Xte, weights, pflip=pflip, pdepol=pdepol, shots=shots)

        with timer_v.timed(f"VQC_metrics_{shots}_{pflip}_{pdepol}"):
            # Fix threshold=0.5 for robustness comparability in sweeps
            m_tr, cm_tr, rep_tr = extended_metrics_from_probs(p_tr, ytr, "train", thr=0.5)
            m_va, cm_va, rep_va = extended_metrics_from_probs(p_va, yva, "val",   thr=0.5)
            m_te, cm_te, rep_te = extended_metrics_from_probs(p_te, yte, "test",  thr=0.5)

            # Annotate and collect
            for m in (m_tr, m_va, m_te):
                m.update(dict(model="VQC", shots=shots, pflip=pflip, pdepol=pdepol))
                rows.append(m)

            # Save confusion matrices per config
            tag = f"vqc_{shots}_{str(pflip).replace('.','p')}_{str(pdepol).replace('.','p')}"
            save_cm_csv(cm_tr, RESULTS/f"metrics/cm_{tag}_train.csv", normalized=False)
            save_cm_csv(cm_tr, RESULTS/f"metrics/cm_{tag}_train_norm.csv", normalized=True)
            save_cm_csv(cm_va, RESULTS/f"metrics/cm_{tag}_val.csv",   normalized=False)
            save_cm_csv(cm_va, RESULTS/f"metrics/cm_{tag}_val_norm.csv",   normalized=True)
            save_cm_csv(cm_te, RESULTS/f"metrics/cm_{tag}_test.csv",  normalized=False)
            save_cm_csv(cm_te, RESULTS/f"metrics/cm_{tag}_test_norm.csv",  normalized=True)

            # Plots (test) per config
            plot_roc(yte, p_te, f"VQC ROC ({cfg})", RESULTS/f"plots/{tag}_roc_test.png")
            plot_pr (yte, p_te, f"VQC PR  ({cfg})", RESULTS/f"plots/{tag}_pr_test.png")

df_vqc = pd.DataFrame(rows)
df_vqc.to_csv(RESULTS/"metrics/noise_sweep_vqc.csv", index=False)
print("VQC sweep rows:", len(df_vqc))
display(df_vqc.head())
J.log("vqc_sweep", "ok", f"Completed VQC sweep; rows={len(df_vqc)}")

# Cell 9 — QSVM sweep (pure cached + MC noisy Nyström; val-optimal threshold; CSV)

In [ ]:
# QSVM robustness sweep: reuse cached pure kernel or build via states; approximate noisy kernels via Nyström
from numpy.linalg import eigh
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix,
    balanced_accuracy_score, matthews_corrcoef, classification_report, average_precision_score,
    precision_recall_curve, roc_curve, auc
)

rows = []
timer_q = PhaseTimer()

# ---------- helpers: shapes & alignment ----------
def _fix_precomputed_shape(K, n_train, name="K"):
    """Ensure precomputed kernel has shape (n_rows, n_train). Transpose/reshape if needed."""
    K = np.asarray(K)
    if K.ndim == 1:
        if K.size % n_train != 0:
            raise ValueError(f"{name}: 1D kernel of size {K.size} not divisible by n_train={n_train}")
        K = K.reshape(-1, n_train)
    elif K.ndim > 2:
        if K.size % n_train != 0:
            raise ValueError(f"{name}: ND kernel with total size {K.size} not divisible by n_train={n_train}")
        K = K.reshape(-1, n_train)
    if K.shape[1] != n_train and K.shape[0] == n_train:
        K = K.T
    if K.shape[1] != n_train:
        raise ValueError(f"{name}: bad shape {K.shape}; expected (*, {n_train})")
    return K

def _align_split_lengths(y_true, n_rows, split_name, tag):
    """
    Make y_true length match n_rows (kernel rows / feature rows).
    If longer, head-slice; if shorter, raise (we cannot create labels).
    """
    y_true = np.asarray(y_true).reshape(-1)
    if len(y_true) == n_rows:
        return y_true
    if len(y_true) > n_rows:
        if 'J' in globals():
            J.log("align", "warn",
                  f"{tag}:{split_name} y length {len(y_true)} > kernel rows {n_rows}; slicing to head {n_rows}.")
        print(f"⚠️  [{tag}::{split_name}] y length {len(y_true)} > rows {n_rows}; slicing y -> {n_rows}")
        return y_true[:n_rows]
    # len(y_true) < n_rows
    raise ValueError(f"{tag}:{split_name} has fewer labels ({len(y_true)}) than rows ({n_rows}). "
                     f"Cannot align by truncating kernel safely. Regenerate kernels or use consistent splits.")

def _plot_roc_pr(y_true, prob, tag_prefix):
    try:
        fpr, tpr, _ = roc_curve(y_true, prob); roc_auc = auc(fpr, tpr)
        plt.figure(); plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
        plt.plot([0,1],[0,1],"--",lw=1); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"QSVM ROC — {tag_prefix}")
        plt.legend(); plt.tight_layout(); plt.savefig(RESULTS/f"plots/{tag_prefix}_roc_test.png", dpi=150); plt.close()
    except Exception as e:
        print(f"⚠️ ROC plot skipped ({tag_prefix}): {e}")
        if 'J' in globals(): J.log("plot","warn",f"ROC skipped {tag_prefix}: {e}")
    try:
        prec, rec, _ = precision_recall_curve(y_true, prob); ap = average_precision_score(y_true, prob)
        plt.figure(); plt.plot(rec, prec, label=f"AP={ap:.3f}")
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"QSVM PR — {tag_prefix}")
        plt.legend(); plt.tight_layout(); plt.savefig(RESULTS/f"plots/{tag_prefix}_pr_test.png", dpi=150); plt.close()
    except Exception as e:
        print(f"⚠️ PR plot skipped ({tag_prefix}): {e}")
        if 'J' in globals(): J.log("plot","warn",f"PR skipped {tag_prefix}: {e}")

def _save_probs(tag_prefix, prob_tr=None, prob_va=None, prob_te=None):
    outdir = RESULTS / "metrics"; outdir.mkdir(parents=True, exist_ok=True)
    if prob_tr is not None: np.save(outdir / f"qsvm_{tag_prefix}_probs_train.npy", prob_tr)
    if prob_va is not None: np.save(outdir / f"qsvm_{tag_prefix}_probs_val.npy",   prob_va)
    if prob_te is not None: np.save(outdir / f"qsvm_{tag_prefix}_probs_test.npy",  prob_te)

def ext_metrics_from_scores(prob, y, split):
    """Return extended metrics dict (thr=0.5) and confusion matrix/report."""
    prob = np.clip(np.asarray(prob).reshape(-1), 1e-6, 1-1e-6)
    pred = (prob >= 0.5).astype(int)
    acc = accuracy_score(y, pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y, pred, average="binary", zero_division=0)
    try: auc_v = roc_auc_score(y, prob)
    except ValueError: auc_v = float("nan")
    try: ap = average_precision_score(y, prob)
    except Exception: ap = float("nan")
    cm = confusion_matrix(y, pred, labels=[0,1])
    tn, fp, fn, tp = cm.ravel()
    tnr = tn / (tn + fp) if (tn + fp) else float("nan")
    bal = balanced_accuracy_score(y, pred)
    mcc = matthews_corrcoef(y, pred) if len(np.unique(y)) == 2 else float("nan")
    rep = classification_report(y, pred, output_dict=True, zero_division=0)
    return dict(split=split, acc=acc, prec=prec, rec=rec, f1=f1, auc=auc_v,
                pr_auc=ap, balanced_acc=bal, specificity=tnr, mcc=mcc, thr=0.5), cm, rep

# ---------- cache fingerprint for noiseless kernels ----------
fp = f"D{D}_N{len(Xtr_s)}_pca17_scaler"
PURE_KERNELS_FILE = RESULTS / f"cache/pure_kernels_{fp}.npz"

pure_cached = load_npz(PURE_KERNELS_FILE)
if pure_cached:
    K_trtr_n = pure_cached["K_trtr_n"]; K_vatr_n = pure_cached["K_vatr_n"]; K_tetr_n = pure_cached["K_tetr_n"]
    print("Loaded pure kernels from cache.")
    J.log("kernel_cache", "ok", f"Loaded cached pure kernels {PURE_KERNELS_FILE.name}")
else:
    K_trtr_n = K_vatr_n = K_tetr_n = None
    J.log("kernel_cache", "warn", "Pure kernels not in cache; will compute on demand.")

with timer_q.timed(f"QSVM_sweep_total_{len(SWEEP)}"):
    for shots, pflip, pdepol in SWEEP:
        print(f"\n[QSVM] shots={shots}, pflip={pflip}, pdepol={pdepol}")
        noiseless = (pflip == 0.0 and pdepol == 0.0)

        if noiseless:
            # Build or load pure kernels
            if K_trtr_n is None:
                with timer_q.timed("pure_states_and_gram"):
                    get_state = make_pure_state_getter()
                    S_tr = states_batch(Xtr_s, get_state)
                    S_va = states_batch(Xva,   get_state)
                    S_te = states_batch(Xte,   get_state)
                    K_trtr = kernel_from_states(S_tr, S_tr)
                    K_vatr = kernel_from_states(S_va, S_tr)
                    K_tetr = kernel_from_states(S_te, S_tr)
                d_tr = np.clip(np.diag(K_trtr), 1e-12, 1.0)
                d_va = np.clip(np.diag(kernel_from_states(S_va, S_va)), 1e-12, 1.0)
                d_te = np.clip(np.diag(kernel_from_states(S_te, S_te)), 1e-12, 1.0)
                with timer_q.timed("pure_normalize"):
                    K_trtr_n = normalize_block(0.5*(K_trtr + K_trtr.T), d_tr, d_tr) + 1e-8*np.eye(len(d_tr))
                    K_vatr_n = normalize_block(K_vatr, d_va, d_tr)
                    K_tetr_n = normalize_block(K_tetr, d_te, d_tr)
                save_npz(PURE_KERNELS_FILE, K_trtr_n=K_trtr_n, K_vatr_n=K_vatr_n, K_tetr_n=K_tetr_n)
                print("Saved pure kernels to cache.")
                J.log("kernel_cache", "ok", f"Saved pure kernels -> {PURE_KERNELS_FILE.name}")

            if np.unique(ytr_s).size < 2:
                J.log("limitation", "warn", "Skip pure QSVM branch — single-class training slice.")
            else:
                # Fit
                with timer_q.timed("pure_svc_fit"):
                    K_trtr_n_fixed = _fix_precomputed_shape(K_trtr_n, K_trtr_n.shape[0], "K_trtr_n")
                    clf = SVC(C=5.0, kernel="precomputed", probability=True, class_weight="balanced", random_state=0)
                    clf.fit(K_trtr_n_fixed, ytr_s)

                # Evaluate splits
                def evalK(K, y_true, split, tag):
                    K_fixed = _fix_precomputed_shape(K, K_trtr_n.shape[0], f"{tag}:{split}")
                    print(f"   shapes -> K:{K_fixed.shape}, y_in:{y_true.shape}", flush=True)
                    # align y to kernel rows if needed
                    y_use = _align_split_lengths(y_true, K_fixed.shape[0], split, tag)
                    t0 = time.perf_counter()
                    try:
                        prob = clf.predict_proba(K_fixed)[:, 1]
                    except Exception:
                        df = clf.decision_function(K_fixed); prob = 1.0/(1.0+np.exp(-df))
                    prob = np.asarray(prob).reshape(-1)
                    # If we had to slice y, slice probs as well to keep parity
                    if prob.shape[0] != y_use.shape[0]:
                        n = min(prob.shape[0], y_use.shape[0])
                        if 'J' in globals():
                            J.log("align", "warn",
                                  f"{tag}:{split} prob len {prob.shape[0]} != y {y_use.shape[0]}; truncating both to {n}.")
                        print(f"⚠️  [{tag}::{split}] prob {prob.shape[0]} != y {y_use.shape[0]}; trunc -> {n}")
                        prob = prob[:n]; y_use = y_use[:n]
                    m, cm, rep = ext_metrics_from_scores(prob, y_use, split)
                    save_cm_csv(cm, RESULTS/f"metrics/cm_qsvm_{tag}_{split}.csv", normalized=False)
                    save_cm_csv(cm, RESULTS/f"metrics/cm_qsvm_{tag}_{split}_norm.csv", normalized=True)
                    timer_q.add(f"pure_eval_{split}", time.perf_counter()-t0)
                    print(f"  [{tag}::{split}] F1={m['f1']:.3f}, AUC={m['auc']:.3f}, PR-AUC={m['pr_auc']:.3f}, ACC={m['acc']:.3f}")
                    return m, prob

                tag = f"pure_{shots}_{str(pflip).replace('.','p')}_{str(pdepol).replace('.','p')}"
                m_tr, prob_tr = evalK(K_trtr_n, ytr_s, "train", tag)
                m_va, prob_va = evalK(K_vatr_n, yva,   "val",   tag)
                m_te, prob_te = evalK(K_tetr_n, yte,   "test",  tag)

                rows += [
                    {**m_tr, "model":"QSVM", "shots":shots, "pflip":pflip, "pdepol":pdepol},
                    {**m_va, "model":"QSVM", "shots":shots, "pflip":pflip, "pdepol":pdepol},
                    {**m_te, "model":"QSVM", "shots":shots, "pflip":pflip, "pdepol":pdepol},
                ]
                _save_probs(tag, prob_tr, prob_va, prob_te)
                _plot_roc_pr(yte, prob_te, f"qsvm_{tag}")

        else:
            # Noisy Nyström approximation
            S_NOISE = max(16, shots // 32) or 16
            rng = np.random.default_rng(123)
            idx_anchor = rng.choice(len(Xtr_s), size=min(M_ANCHORS, len(Xtr_s)), replace=False)
            A = Xtr_s[idx_anchor]

            with timer_q.timed("noisy_blocks_states"):
                K_MM, K_trM, K_vaM, K_teM, d_M, d_tr, d_va, d_te = mc_blocks_via_states(
                    Xtr_s, Xva, Xte, A, pflip, pdepol, S=S_NOISE, seed=123
                )
            with timer_q.timed("noisy_normalize"):
                def norm(K, da, db):
                    da = np.where(da <= 1e-12, 1e-12, da)
                    db = np.where(db <= 1e-12, 1e-12, db)
                    return np.clip(K, 0.0, 1.0) / (np.sqrt(np.outer(da, db)) + 1e-12)
                K_MM_n  = norm(0.5*(K_MM + K_MM.T), d_M, d_M) + 1e-8*np.eye(len(d_M))
                K_trM_n = norm(K_trM, d_tr, d_M)
                K_vaM_n = norm(K_vaM, d_va, d_M)
                K_teM_n = norm(K_teM, d_te, d_M)

            with timer_q.timed("noisy_features"):
                w, V = eigh(K_MM_n)
                Winv_sqrt = V @ np.diag(1.0/np.sqrt(np.clip(w + 1e-6, 1e-12, None))) @ V.T
                Phi_tr = K_trM_n @ Winv_sqrt
                Phi_va = K_vaM_n @ Winv_sqrt
                Phi_te = K_teM_n @ Winv_sqrt

            if np.unique(ytr_s).size < 2:
                J.log("limitation", "warn", "Skip noisy QSVM branch — single-class training slice.")
            else:
                with timer_q.timed("noisy_linear_svc_fit"):
                    clf = SVC(C=5.0, kernel="linear", probability=True, class_weight="balanced", random_state=0)
                    clf.fit(Phi_tr, ytr_s)

                def eval_feat(F, y_true, split, tag):
                    print(f"   shapes -> Φ:{F.shape}, y_in:{y_true.shape}", flush=True)
                    y_use = _align_split_lengths(y_true, F.shape[0], split, tag)
                    t0 = time.perf_counter()
                    try:
                        prob = clf.predict_proba(F)[:, 1]
                    except Exception:
                        df = clf.decision_function(F); prob = 1.0/(1.0+np.exp(-df))
                    prob = np.asarray(prob).reshape(-1)
                    if prob.shape[0] != y_use.shape[0]:
                        n = min(prob.shape[0], y_use.shape[0])
                        if 'J' in globals():
                            J.log("align", "warn",
                                  f"{tag}:{split} prob len {prob.shape[0]} != y {y_use.shape[0]}; truncating both to {n}.")
                        print(f"⚠️  [{tag}::{split}] prob {prob.shape[0]} != y {y_use.shape[0]}; trunc -> {n}")
                        prob = prob[:n]; y_use = y_use[:n]
                    m, cm, rep = ext_metrics_from_scores(prob, y_use, split)
                    save_cm_csv(cm, RESULTS/f"metrics/cm_qsvm_{tag}_{split}.csv", normalized=False)
                    save_cm_csv(cm, RESULTS/f"metrics/cm_qsvm_{tag}_{split}_norm.csv", normalized=True)
                    timer_q.add(f"noisy_eval_{split}", time.perf_counter()-t0)
                    print(f"  [{tag}::{split}] F1={m['f1']:.3f}, AUC={m['auc']:.3f}, PR-AUC={m['pr_auc']:.3f}, ACC={m['acc']:.3f}")
                    return m, prob

                tag = f"noisy_{shots}_{str(pflip).replace('.','p')}_{str(pdepol).replace('.','p')}_A{len(A)}_S{S_NOISE}"
                m_tr, prob_tr = eval_feat(Phi_tr, ytr_s, "train", tag)
                m_va, prob_va = eval_feat(Phi_va, yva,   "val",   tag)
                m_te, prob_te = eval_feat(Phi_te, yte,   "test",  tag)

                rows += [
                    {**m_tr, "model":"QSVM", "shots":shots, "pflip":pflip, "pdepol":pdepol, "anchors": int(len(A)), "S_NOISE": int(S_NOISE)},
                    {**m_va, "model":"QSVM", "shots":shots, "pflip":pflip, "pdepol":pdepol, "anchors": int(len(A)), "S_NOISE": int(S_NOISE)},
                    {**m_te, "model":"QSVM", "shots":shots, "pflip":pflip, "pdepol":pdepol, "anchors": int(len(A)), "S_NOISE": int(S_NOISE)},
                ]
                _save_probs(tag, prob_tr, prob_va, prob_te)
                _plot_roc_pr(yte, prob_te, f"qsvm_{tag}")

# finalize
df_qsvm = pd.DataFrame(rows)
df_qsvm.to_csv(RESULTS/"metrics/noise_sweep_qsvm.csv", index=False)
print("QSVM sweep rows:", len(df_qsvm)); display(df_qsvm.head())
if 'J' in globals(): J.log("qsvm_sweep", "ok", f"Completed QSVM sweep; rows={len(df_qsvm)}")

# Cell 10 — pivots & JSON run report (with real timers)

In [ ]:
# Summarize test split performance; persist pivot tables and a JSON timing/config report
METRICS_DIR = RESULTS / "metrics"

# Pivots (test split only)
if len(df_vqc) == 0 or len(df_qsvm) == 0:
    J.log("summary", "warn", "One of the sweep frames is empty; pivots may be incomplete.")
pv_vqc  = (df_vqc[df_vqc["split"]=="test"]
           .pivot_table(index=["shots","pflip","pdepol"], values=["f1","auc","acc","pr_auc","balanced_acc","specificity","mcc"], aggfunc="mean")
           .sort_values("f1", ascending=False))
pv_qsvm = (df_qsvm[df_qsvm["split"]=="test"]
           .pivot_table(index=["shots","pflip","pdepol"], values=["f1","auc","acc","pr_auc","balanced_acc","specificity","mcc"], aggfunc="mean")
           .sort_values("f1", ascending=False))

pv_vqc.to_csv(METRICS_DIR/"pivot_vqc_test.csv")
pv_qsvm.to_csv(METRICS_DIR/"pivot_qsvm_test.csv")

# Run report with timers & configuration
run_report = {
    "config": {
        "D": int(D), "MAX_TR": int(len(Xtr_s)), "M_ANCHORS": int(M_ANCHORS),
        "sweep_size": int(len(SWEEP)),
        "shots_grid": shots_grid, "pflip_grid": pflip_grid, "pdepol_grid": pdepol_grid,
    },
    "timing": {
        "VQC": {k: pretty_seconds(v) for k,v in PhaseTimer().to_dict().items()},  # placeholder if not used
        "QSVM": {k: pretty_seconds(v) for k,v in PhaseTimer().to_dict().items()}, # placeholder if not used
        "VQC_actual": {k: pretty_seconds(v) for k,v in timer_v.to_dict().items()},
        "QSVM_actual": {k: pretty_seconds(v) for k,v in timer_q.to_dict().items()},
    }
}
with open(METRICS_DIR/"noise_sweep_run_report.json", "w", encoding="utf-8") as f:
    json.dump(run_report, f, indent=2)

print("\n=== Run timing summary ===")
for k, v in timer_v.to_dict().items():
    print(f"VQC:{k:>30}  {pretty_seconds(v)}")
for k, v in timer_q.to_dict().items():
    print(f"QSVM:{k:>29}  {pretty_seconds(v)}")

print("\nSaved:")
print(" - results/metrics/noise_sweep_vqc.csv")
print(" - results/metrics/noise_sweep_qsvm.csv")
print(" - results/metrics/pivot_vqc_test.csv")
print(" - results/metrics/pivot_qsvm_test.csv")
print(" - results/metrics/noise_sweep_run_report.json")

# Journal: roll-up of warnings/failures
issues = []
for e in J.events:
    if e["status"] in ("warn","fail"):
        issues.append(f"- [{e['step']}] {e['message']}")
rollup = "No warnings or failures." if not issues else "Issues observed:\n" + "\n".join(issues)
print("\n=== RUN SUMMARY ===\n" + rollup)

ts = time.strftime("%Y%m%d_%H%M%S")
J.save(RESULTS/"logs"/f"noise_robustness_{ts}")
(RESULTS/"logs"/f"noise_robustness_{ts}_summary.txt").write_text(rollup, encoding="utf-8")